In [2]:
from collections import defaultdict
import torch.optim as optim
import torch.nn as nn
import torch
import torchtext
from transformers.tokenization_bert import BertTokenizer
from transformers.modeling_bert import BertForSequenceClassification

In [3]:
# Set params
device = 'cuda'
pretrained_weights = 'bert-base-uncased' # specify type of BERT we want

# Load BERT

In [4]:
# load BERT tokenizer and classifier
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
BERT = BertForSequenceClassification.from_pretrained(pretrained_weights)

100%|██████████| 440473133/440473133 [00:09<00:00, 44950191.53B/s]


In [5]:
# Sent BERT to GPU
BERT.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [6]:
input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")]).to(device)
BERT(input_ids)

(tensor([[-0.2726,  0.2188]], device='cuda:0', grad_fn=<AddmmBackward>),)

# Load IMDb

In [7]:
# set up fields
TEXT = torchtext.data.Field(
    lower=True, 
    fix_length = 512,
    batch_first=True, 
    truncate_first=True,
    tokenize = tokenizer.tokenize, 
    pad_token='[PAD]',
    unk_token='[UNK]',
    init_token='[CLS]',
    eos_token='[SEP]',
)
LABEL = torchtext.data.Field(sequential=False)

In [8]:
# make splits for data
train, test = torchtext.datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:04<00:00, 18.7MB/s]


In [9]:
# build the vocabulary
TEXT.build_vocab(train)
LABEL.build_vocab(train)

itos = [token for token in tokenizer.vocab]
stoi = {token: index for index, token in enumerate(tokenizer.vocab)}

TEXT.vocab.itos = itos
TEXT.vocab.stoi = stoi

In [13]:
# make iterator for splits
train_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train, test), batch_size=16, device = device)

# Train BERT

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(BERT.parameters())

In [15]:
for epoch in range(10):
    for batch in train_iter:
        # format labels
        label = batch.label - 1
        optimizer.zero_grad()
        output = BERT(batch.text)[0]
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

tensor([[  101,  1045,  2001,  ...,     0,     0,     0],
        [  101,  2425,  1996,  ...,     0,     0,     0],
        [  101,  1045,  2228,  ...,     0,     0,     0],
        ...,
        [  101,  2051,  2000,  ..., 13905,  1012,   102],
        [  101,  1045,  3422,  ...,     0,     0,     0],
        [  101,  1045,  2253,  ...,     0,     0,     0]], device='cuda:0')


RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 11.17 GiB total capacity; 10.74 GiB already allocated; 72.81 MiB free; 52.00 MiB cached)